In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clients = pd.read_csv('clients.csv')

In [ ]:
loans = pd.read_csv('loans.csv')

In [ ]:
payments = pd.read_csv('payments.csv')

In [ ]:
clients.head()

In [ ]:
loans.head()

In [ ]:
payments.head()

# Section 1 Ans

### Q1. Give the 5 client IDs with the highest mean payment amount

In [ ]:
agg_loan_id = (payments.groupby(['loan_id'])[['payment_amount']].mean()).sort_values(by = 'payment_amount', ascending = False)

In [ ]:
top5_loan_id = list(agg_loan_id.head().index)

In [ ]:
top5_loan_id

In [ ]:
top5_client_id = loans[loans['loan_id'].isin(top5_loan_id)]['client_id'].values

In [ ]:
top5_client_id

### Q1.Ans = [49545, 35214, 29841, 26695, 38537]

### Q2. How many unique loans have been given out to clients who joined prior to 2001?

In [ ]:
clients['joined'] = clients['joined'].astype('datetime64[ns]')

In [ ]:
clients['joined_year'] = clients['joined'].map(lambda x: x.strftime('%Y'))

In [ ]:
clients = clients.sort_values(by = 'joined_year', ascending = True)

In [ ]:
clients

#### There are only 3 clients prior to 2001. There clients ids are 26945,39384,42320

In [ ]:
cli_id_2001 = [26945,39384,42320]

In [ ]:
uni_cli_id = loans[loans['client_id'].isin(cli_id_2001)]

In [ ]:
uni_cli_id

In [ ]:
uni_loans = uni_cli_id.loan_id.unique()

In [ ]:
len(uni_loans)

### Q2.Ans 49 unique loans have been given out to clients who joined prior to 2001

### Q3. What is the mean number of payments missed by clients with a credit score of less than 700 and who have missed more than 50 payments?

# Section 2 Ans

### Q1. Create a histogram of the payment amounts. Briefly describe the distribution.

In [ ]:
sns.set(rc={"figure.figsize": (15, 8)})
pay = payments['payment_amount']
ax = sns.distplot(pay,bins=15)
plt.xlabel("Payment Amount", fontsize= 15)
plt.title("Histogram of Payments", fontsize= 15)

In [ ]:
 payments['payment_amount'].describe()

In [ ]:
 payments['payment_amount'].mode()

#### Here the distribution in not normal but positively skewed because it’s long tail is on the positive direction on the number line. Usally in a right skewed distribution, the mean is usually to the right of the median.

#### Frequency of the loan amount below 1500 is greater than that of the loan amount above 1500

#### KDE plot shows the probability density at different values in a continuous variable.

### Q2. Produce a line plot the cumulative sum of the number of clients by year.

In [ ]:
clients

In [ ]:
df = clients['joined_year'].value_counts().sort_index().rename_axis('Year').reset_index(name='Count')

In [ ]:
df.set_index('Year', inplace=True)

In [ ]:
plt.xlabel('Year') 
plt.ylabel('Cum Sum') 
plt.title('Cumulative sum of the number of clients by year')
plt.plot(df.cumsum())
plt.show() 

### Q3. Produce a scatter plot of the percentage of payments missed in december for each year in the dataset.

# Section 3 - Modelling

In [ ]:
data = pd.read_csv('test_diabetes.csv',sep=';')

In [ ]:
data

Data preprocessing

In [ ]:
data.isnull().sum()

In [ ]:
data = data.dropna()

Dataset contains missing values. If there exist missing values then it is better to either delete or replace them by mean/median/mode from the data. 

In [ ]:
data['Outcome'] = data['Outcome'].replace({'N':0,'Y':1})
data['Insulin'] = data['Insulin'].replace({'Zero':0})

In [ ]:
data

In [ ]:
col = list(data.columns)

convert column values to numeric

In [ ]:
for co in col:
    data[co] = pd.to_numeric(data[co])

In [ ]:
data.shape

Splitting the dataset into train and test set.

In [ ]:
train = data[:400]
test = data[400:503]

Feature Scaling

In [ ]:
y_train = np.asarray(train['Outcome'])
x_train = np.asarray(train.drop('Outcome',1))
y_test = np.asarray(test['Outcome'])
x_test = np.asarray(test.drop('Outcome',1))

Normalization

In [ ]:
means = np.mean(x_train, axis=0)
stds = np.std(x_train, axis=0)
x_train = (x_train - means)/stds
x_test = (x_test - means)/stds

Model selection

In [ ]:
def train(x_train,y_train):
    
    log=LogisticRegression(random_state=0)
    log.fit(x_train,y_train)


    tree=DecisionTreeClassifier(criterion='entropy',random_state=0)
    tree.fit(x_train,y_train)

    rf= RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
    rf.fit(x_train,y_train)

    #model accuracy
    print('Logistic accuracy:',log.score(x_train,y_train))
    print('Decision Tree accuracy:',tree.score(x_train,y_train))
    print('Random forest accuracy:',rf.score(x_train,y_train))
    return log,tree,rf

Training the model

In [ ]:
train(x_train,y_train)

We should use Logistic regression model because random forest and decision trees models shows overfitting since any accuracy above 90 is considered to be overfitting.

In [ ]:
def test(x_test,y_test):
    log=LogisticRegression(random_state=0)
    log.fit(x_train,y_train)
    accuracy = log.score(x_test, y_test)
    return accuracy

Testing the model

In [ ]:
test(x_test,y_test)